In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize
import tensorflow as tf 


Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    
    if ds.PatientPosition != 'PA' and ds.PatientPosition != 'AP':
        print('The image position is not valid!\n')
        return None
    
    if ds.BodyPartExamined != 'CHEST':
        print('The examined body part is not chest!\n')
        return None
        
    if ds.Modality != 'DX':
        print('The image type is not valid!\n')
        return None
    
    if int(ds.PatientAge) < 20 and int(ds.PatientAge) > 80:
        print('The patient age is out of the intended range! \n')
        return None
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # https://knowledge.udacity.com/questions/308593
    proc_img = (img - img_mean)/img_std
    proc_img =  resize(proc_img, img_size, anti_aliasing=True)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    
    with open(model_path, 'r') as json_file:
        saved_model = json_file.read()
        
    model = tf.keras.models.model_from_json(saved_model)
    model.summary()
    
    model.load_weights(weight_path)
    
#     print(f'Model Loaded: {model}')
    
    return model


# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    result = model.predict(img)  
    predict=result[0]
    
    prediction='No pneumonia'
    
    if(predict>thresh):
        prediction='Pneumonia'
        
    return prediction 

# https://knowledge.udacity.com/questions/188296

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "./my_model.json" 
weight_path = "./xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.37

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,np.mean(img),np.std(img),IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(f'Prediction: {pred}' + '\n')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)              